In [1]:
import pandas as pd
import numpy as np

from ast import literal_eval
from collections import Counter
from tqdm.auto import tqdm
import warnings

pd.set_option('display.max_colwidth', 1000)
warnings.filterwarnings("ignore")
tqdm.pandas()

In [2]:
import csv

def combine_csv_columns(file1, file2, output_file):
    with open(file1, 'r', newline='') as f1, open(file2, 'r', newline='') as f2, open(output_file, 'w', newline='') as output:
        reader1 = csv.reader(f1)
        reader2 = csv.reader(f2)

        writer = csv.writer(output)

        for row1, row2 in zip(reader1, reader2):
            combined_row = row1 + row2
            writer.writerow(combined_row)

#combine_csv_columns('ratings.csv', 'cycle_data.csv', 'combined_columns.csv')

In [2]:
df = pd.read_csv('combined_columns.csv')
df.head()

,foodnumber,User,Rating,Time Taken (s),Total Distance (km),Average Speed (km/h),Total Calories Burned (cal)
0,0,111,1,2081,17.91,30.99,613.64
1,1,112,4,2476,10.94,15.90,247.50
2,2,113,3,3563,44.99,45.45,1041.50
3,3,114,4,1138,17.43,55.13,372.60
4,4,115,4,377,4.58,43.73,105.40


In [3]:
#top 5 most calories burnt
sorted_df = df.sort_values(by='Total Calories Burned (cal)', ascending=False)

top_5_users = sorted_df.head(5)
top_5_users

,foodnumber,User,Rating,Time Taken (s),Total Distance (km),Average Speed (km/h),Total Calories Burned (cal)
85,85,996,4,3459,46.55,48.44,1375.90
139,139,1420,5,3458,56.91,59.24,1350.10
143,143,1503,3,3294,35.83,39.16,1318.46
63,63,774,2,3331,33.04,35.71,1311.70
133,133,1414,3,3467,38.13,39.59,1283.59


In [5]:
import plotly.graph_objects as go
import pandas as pd

calories_burned = df['Total Calories Burned (cal)']
fig = go.Figure()
fig.add_trace(go.Histogram(x=calories_burned, marker_color='skyblue', opacity=0.75))
fig.update_layout(
    shapes=[
        dict(
            type='line',
            xref='x',
            yref='y',
            x0=600,
            y0=0,
            x1=600,
            y1=30,
            line=dict(color='red', width=2)
        )
    ]
)
fig.add_annotation(
    x=600,
    y=30,
    text='Recommended: 600 Calories to burn in one cycling activity',
    showarrow=True,
    arrowhead=1,
    arrowcolor='black',
    arrowwidth=2,
    ax=20,
    ay=-30,
    bgcolor='lightgray',
    opacity=0.8
)
fig.update_layout(
    title='Histogram of Total Calories Burned',
    xaxis_title='Total Calories Burned (cal)',
    yaxis_title='Frequency',
    bargap=0.05
)

fig.show()

In [7]:
import dash
from dash import dcc, html
import dash_table
from dash.dependencies import Input, Output

df_sorted = df.sort_values(by='Total Calories Burned (cal)', ascending=False)

df_sorted['Rank'] = range(1, len(df_sorted) + 1)

top_10_users = df_sorted.head(10)
top_10_users = top_10_users.drop(columns=['foodnumber', 'Rating'])


app = dash.Dash(__name__)


app.layout = html.Div([
    html.H1('Leaderboard', style={'textAlign': 'center', 'color': '#333333'}),
    dash_table.DataTable(
        id='leaderboard-table',
        columns=[
            {'name': 'Rank', 'id': 'Rank'},
            {'name': 'User', 'id': 'User'},
            {'name': 'Time Taken (s)', 'id': 'Time Taken (s)'},
            {'name': 'Total Distance (km)', 'id': 'Total Distance (km)'},
            {'name': 'Average Speed (km/h)', 'id': 'Average Speed (km/h)'},
            {'name': 'Total Calories Burned (cal)', 'id': 'Total Calories Burned (cal)'}
        ],
        data=top_10_users.to_dict('records'),
        style_table={'overflowX': 'auto', 'border': '2px solid #999999', 'border-radius': '5px'},
        style_header={'backgroundColor': '#f2f2f2', 'fontWeight': 'bold', 'color': '#333333'},
        style_data={'whiteSpace': 'normal', 'height': 'auto', 'color': '#333333'},
        style_cell={'textAlign': 'center', 'fontFamily': 'Arial, sans-serif'},
        style_as_list_view=True,
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)